In [1]:
##IMPORT 

import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST
import matplotlib.pyplot as plt
from patchify import patchify


In [2]:
##PATCHFY FUNCTION & POSITIONAL_EMB

def patching_func(images,patch_size):

    n,c,h_iamge,w_image=images.shape
    n_patch=0
    n_patch_per_img = int((h_iamge/patch_size)**2)
    patch_vector=int(patch_size**2)
    images=images.squeeze()

    total_patches = np.zeros((n,n_patch_per_img,patch_vector))
 
    for idx,image in enumerate(images):
        patching = patchify(image, (patch_size,patch_size),step=4) # split image into 7x7 small 4x4 patches.patchify(image, (3, 3), step=1)
        for i in range(patching.shape[0]):
            for j in range (patching.shape[1]):
                single_patch=patching[i,j,:,:]
                total_patches[idx,n_patch,:]=single_patch.flatten()
                n_patch=n_patch+1
                if n_patch==49:
                    n_patch=0
    return total_patches

def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result

In [3]:
#IMPORT DATA
# 
transform = ToTensor()
train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
test_set = MNIST(root='./../datasets', train=False, download=False, transform=transform)

train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

In [6]:
#DATA TO PATCH
for batch in train_loader:
    images, targets = batch 

#plt.imshow(images[0].squeeze())      #1x28x28--> 28x28
images=images.numpy()

print(images.shape)
print(patching_func(images,4).shape)
patches=torch.tensor(patching_func(images,4))
print(patches.shape)
print(patches.float().dtype)

(96, 1, 28, 28)
(96, 49, 16)
torch.Size([96, 49, 16])
torch.float32


In [ ]:
token_dim=8
patch_size=4

class_token = nn.Parameter(torch.rand(96,1, token_dim))
linear_map=nn.Linear(patch_size**2,token_dim)

print(class_token.shape)
print(patches.dtype)
print(patches.shape)

linear_emb = linear_map(patches.float())
print(linear_emb.shape)

print(class_token.shape)

tokens = torch.cat((class_token,linear_emb),dim=1)
print(tokens.shape)

In [8]:
token_dim=8
layer_norms = nn.ModuleList([nn.LayerNorm(token_dim) for _ in range(2)])
print(layer_norms[0](tokens).shape)


torch.Size([96, 50, 8])


In [68]:
# CLASSES VIT

class ViT (nn.Module):
    def __init__(self, images_shape, patch_size=4, t_blocks=2, token_dim=8, n_heads=2, output_dim=10, mlp_layer_size=8):
        super().__init__()
        
        n_images,c,h_image,w_image=images_shape

        self.patch_size = patch_size
        self.t_blocks   = t_blocks
        self.n_heads    = n_heads
        self.token_dim  = token_dim
        self.output_dim = output_dim
        self.mlp_layer_size = mlp_layer_size

        linear_map          =   nn.Linear(self.patch_size**2,token_dim)
        self.class_token    =   nn.Parameter(torch.rand(n_images, 1, self.token_dim))
        self.blocks         =   nn.ModuleList([ViTBlock(token_dim, n_heads, mlp_layer_size) for _ in range(t_blocks)])
        self.linear_classifier  = nn.Linear(self.token_dim, output_dim)
        output_pr           =   nn.Softmax(dim=-1)

    def forward(self, images):

        patches     = patching_func(images, self.patch_size)
        linear_emb  = self.linear_map(patches)
        tokens      = torch.cat((self.class_token,linear_emb),dim=1)
        out         = tokens

        for block in self.blocks:
            out = block(out)
            
        out = out[:, 0, :]
        out = self.output_pr(self.linear_classifier(out))
        return out 


class ViTBlock(nn.Module):

    def __init__(self, token_dim, num_heads, mlp_layer_size):
        super().__init__() 

        self.layer_norms = nn.ModuleList([nn.LayerNorm(self.token_dim) for _ in range(2)])    
        self.msa = MSA_Module(token_dim, num_heads)
        act_layer = nn.GELU()
        self.mlp_layer_size = mlp_layer_size

        self.mlp = nn.Sequential(
            nn.Linear(token_dim, self.mlp_layer_size * token_dim), act_layer,
            nn.Linear(self.mlp_layer_size * token_dim, token_dim)
            )

    def forward(self, x):
        out = x + self.msa(self.layer_norms[0](x))
        out = self.layer_norms[1](out)+ self.mlp(self.layer_norms[1](out))
        return out


class MSA_Module(nn.Module):
    def __init__(self, token_dim, n_heads=2):
        self.n_heads    = n_heads
        self.token_dim  = token_dim

        self.q_layers   = nn.ModuleList([nn.Linear(token_dim,token_dim) for _ in range(self.n_heads)])
        self.k_layers   = nn.ModuleList([nn.Linear(token_dim,token_dim) for _ in range(self.n_heads)])
        self.v_layers   = nn.ModuleList([nn.Linear(token_dim,token_dim) for _ in range(self.n_heads)])
        softmax         = nn.Softmax()

    def forward (self, tokens):


        out = np.zeros((n, ,50 ,token_dim))

        for token in tokens:   # 128 batch. each of 50x8, token size : 50x8

            out = []
            
            for head in range(self.n_heads):        # number of heads : 2
                self.q_linear = self.q_layers[head]  
                self.k_linear = self.k_layers[head]
                self.v_linear = self.v_layers[head]

                q  = q_linear(token)
                k  = k_linear(token)
                v  = v_linear(token)

                attention = nn.Softmax(q@k.T)



                






        

In [ ]:

class MSA_Module(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MSA_Module, self).__init__()
        self.d = d
        self.n_heads = n_heads
        assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"
        d_head = int(d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)
    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]
                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)
                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

In [69]:
def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
    test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")
    model = ViT((1, 28, 28), patch_size=4, t_blocks=2, token_dim=64, n_heads=2, output_dim=10).to(device)
    N_EPOCHS = 5
    LR = 0.005

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()
    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(test_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)
        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")


In [ ]:

if __name__ == '__main__':
    main()